In [1]:
import os, os.path
os.chdir('../')

In [2]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [4]:
import jieba 
import jieba.analyse
import pickle as pkl

### Read the Data

In [5]:
data = pd.read_csv('data/processed_data.csv')

In [6]:
data.head()

,Unnamed: 0,id,text1,text2,label
0,0,1,﻿怎么更改花呗手机号码,我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号,1
1,1,2,也开不了花呗，就这样了？完事了,真的嘛？就是花呗付款,0
2,2,3,花呗冻结以后还能开通吗,我的条件可以开通花呗借款吗,0
3,3,4,如何得知关闭借呗,想永久关闭借呗,0
4,4,5,花呗扫码付钱,二维码扫描可以用花呗吗,0


### Vectorization

In [7]:
text1 = list(data['text1'])
text2 = list(data['text2'])
labels = list(data['label'])
assert len(text1) == len(text2)
texts = text1 + text2

In [8]:
tokens = []
for text in texts:
    for sentence in re.findall(r'\w+', text):
        for i in range(len(sentence)-1):
            word = sentence[i:i+2]
            tokens.append(word)
counter_2 = Counter(tokens)
most_common_2 = [word[0] for word in counter_2.most_common(30)]

In [9]:
tokens_3 = []
texts = data['text1']
for text in texts:
    for sentence in re.findall(r'\w+', text):
        for i in range(len(sentence)-2):
            word = sentence[i:i+3]
            tokens_3.append(word)
            
counter_3 = Counter(tokens_3)
most_common_3 = [word[0] for word in counter_3.most_common(30)]

In [13]:
word = most_common_2[0]
word

'花呗'

In [14]:
keywords = most_common_2 + most_common_3
word_vector = np.zeros((len(text1), 2*len(keywords)))
for i, word in enumerate(keywords):
    ip = i + len(keywords)
    for j in range(len(word_vector)):
        if word in text1[j]:
            word_vector[j, i] = 1
        if word in text2[j]:
            word_vector[j, ip] = 1

## keywords extraction using jieba

In [78]:
keywords

['花呗',
 '借呗',
 '怎么',
 '什么',
 '还款',
 '可以',
 '蚂蚁',
 '为什',
 '额度',
 '我的',
 '蚁借',
 '分期',
 '开通',
 '用花',
 '支付',
 '的花',
 '呗还',
 '不能',
 '没有',
 '使用',
 '呗分',
 '付款',
 '呗的',
 '不了',
 '付宝',
 '个月',
 '呗怎',
 '蚁花',
 '么我',
 '呗额',
 '为什么',
 '蚂蚁借',
 '蚁借呗',
 '用花呗',
 '的花呗',
 '我的花',
 '呗分期',
 '花呗分',
 '支付宝',
 '呗还款',
 '花呗还',
 '呗怎么',
 '蚂蚁花',
 '蚁花呗',
 '呗额度',
 '什么我',
 '通花呗',
 '开通花',
 '呗可以',
 '花呗的',
 '借呗还',
 '银行卡',
 '提前还',
 '了花呗',
 '花呗怎',
 '花呗付',
 '花呗额',
 '可以用',
 '什么时',
 '么时候']

In [136]:
def full_cut(sentence):
    return ('/'.join(jieba.cut(sentence,cut_all = True))).split('/')
def extract_key_words(sentence,topK):
    return jieba.analyse.extract_tags(sentence,topK,allowPOS=())
def get_all_words(sentences):
    words = []
    for i in range(len(sentences)):
        cutted_sentence = full_cut(sentences[i])
        words.extend(cutted_sentence)
    return words
def get_key_words(sentences):
    words = []
    for i in range(len(sentences)):
        cutted_sentence = extract_key_words(sentences[i],5)
        words.extend(cutted_sentence)
    return words

In [101]:
#dictionary update
added_word = ['花呗','借呗','我的','怎么']
for i in added_word:
    jieba.add_word(i)


In [102]:
#get full-cut mode words from jieba method
all_words = get_all_words(texts)

In [138]:
#get keywords from jieba method
key_words = get_key_words(texts)

In [103]:
#count and clean
counter_full= Counter(all_words)
counter_full.pop('',None)

35633

In [141]:
#count and clean
counter_key= Counter(key_words)
counter_key.pop('',None)

In [142]:
most_common_key = [word[0] for word in counter_full.most_common(60)]
most_common_key

['花呗',
 '借呗',
 '还款',
 '蚂蚁',
 '怎么',
 '我的',
 '额度',
 '可以',
 '为什么',
 '分期',
 '开通',
 '不能',
 '使用',
 '支付宝',
 '不了',
 '付款',
 '逾期',
 '什么',
 '支付',
 '提前',
 '没有',
 '多少',
 '退款',
 '银行卡',
 '收款',
 '关闭',
 '自动',
 '余额',
 '申请',
 '时候',
 '还清',
 '怎么办',
 '账单',
 '淘宝',
 '显示',
 '临时',
 '如何',
 '利息',
 '现在',
 '手续费',
 '信用卡',
 '绑定',
 '账号',
 '取消',
 '信用',
 '扣款',
 '商家',
 '支持',
 '网商贷',
 '收钱',
 '影响',
 '怎样',
 '恢复',
 '这个',
 '冻结',
 '多久',
 '最低',
 '我用',
 '哪里',
 '才能']

In [106]:
#find the most common counts
most_common_full = [word[0] for word in counter_full.most_common(60)]

In [107]:
most_common_full

['花呗',
 '借呗',
 '我',
 '了',
 '吗',
 '怎么',
 '的',
 '什么',
 '还',
 '还款',
 '可以',
 '蚂蚁',
 '为什么',
 '额度',
 '我的',
 '用',
 '分期',
 '是',
 '开通',
 '支付',
 '不能',
 '没有',
 '钱',
 '使用',
 '能',
 '有',
 '在',
 '付款',
 '不了',
 '支付宝',
 '多少',
 '能用',
 '逾期',
 '会',
 '提前',
 '显示',
 '银行',
 '月',
 '现在',
 '信用',
 '手机',
 '后',
 '时候',
 '银行卡',
 '要',
 '退款',
 '收款',
 '自动',
 '申请',
 '余额',
 '一个',
 '关闭',
 '和',
 '到',
 '这个',
 '么',
 '已经',
 '如何',
 '怎么办',
 '不']

In [110]:
extract_key_words(text1[0],10)

['花呗', '手机号码', '更改', '怎么']

In [150]:
def build_train(keywords,text1,text2):
    word_vector = np.zeros((len(text1),embedding_size))
    for i,word in enumerate(keywords):
        ip = i+len(keywords)
        for j in range(len(word_vector)):
            if word in extract_key_words(text1[j],5):
                word_vector[j,i] = 1
            if word in extract_key_words(text2[j],5):
                word_vector[j,ip] = 1
    return word_vector


In [152]:
def build_train(keywords,text1,text2):
    word_vector = np.zeros((len(text1),embedding_size))
    for i,word in enumerate(keywords):
        ip = i+len(keywords)
        for j in range(len(word_vector)):
            if word in text1[j]:
                word_vector[j,i] = 1
            if word in text1[j]:
                word_vector[j,ip] = 1
    return word_vector

In [153]:
keywords = most_common_key
embedding_size = 120

word_vector = build_train(keywords,text1,text2)

## Training

In [146]:
data = pd.DataFrame(word_vector)

In [147]:
input_dim = word_vector.shape[1]

In [148]:
data_keras = np.array(data)
labels = np.array(labels)

In [154]:
model = Sequential()
model.add(Dense(64, input_dim=input_dim, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(data_keras, labels, epochs=100, batch_size=10)
scores = model.evaluate(data_keras, labels)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/100
39312/39312 [==============================] - 7s - loss: 0.5204 - acc: 0.7823        